<a href="https://colab.research.google.com/github/Romainkul/Datathon_KUL/blob/main/Datathon_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import polars as pl
import requests
from zipfile import ZipFile

# URL to the dataset and base directory where the dataset should reside.
url = "https://kuleuven-datathon-2025.s3.eu-central-1.amazonaws.com/Posters.zip"
base_dir = r"C:\Users\romai\Desktop\datathon\Posters"
zip_path = r"C:\Users\romai\Desktop\datathon\Posters.zip"

# Ensure the base directory exists.
os.makedirs(base_dir, exist_ok=True)

# Download the ZIP file only if it doesn't already exist.
if not os.path.exists(zip_path):
    print("Downloading dataset...")
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for a bad status.
    with open(zip_path, "wb") as f:
        f.write(response.content)
    print("Download complete.")
else:
    print("Zip file already exists. Skipping download.")

# Check if the dataset has been extracted by looking for the 'raw' folder.
raw_dir = os.path.join(base_dir, 'raw')
if not os.path.exists(raw_dir):
    print("Extracting dataset...")
    with ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(base_dir)
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

# Define the paths for the processed images and texts.
processed_dir = os.path.join(base_dir, 'preprocessed')
texts_dir = os.path.join(base_dir, 'texts')

# Map each raw subfolder to the languages that are expected to be available.
folder_language_map = {
    'double_page_german_dutch': ['german', 'dutch'],
    'multipart_dutch': ['dutch'],
    'multipart_french': ['french'],
    'single_patch_dutch': ['dutch'],
    'triple_page_german_dutch_french': ['german', 'dutch', 'french'],
}

# List of valid image extensions.
image_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff')

# Initialize lists for Polars DataFrame
raw_paths, german_processed, dutch_processed, french_processed = [], [], [], []
german_texts, dutch_texts, french_texts = [], [], []

# Loop through each subfolder in the raw directory.
for folder in os.listdir(raw_dir):
    folder_path = os.path.join(raw_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    # Determine which languages are expected for this folder.
    languages = folder_language_map.get(folder, [])

    # Loop through each file in the raw subfolder.
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(image_extensions):
            continue

        # Full path to the raw image file.
        raw_file_path = os.path.join(folder_path, filename)

        # Prepare paths and texts for available languages
        processed_paths = {lang: os.path.join(processed_dir, lang, filename) if lang in languages else '' 
                           for lang in ['german', 'dutch', 'french']}
        
        texts = {}
        base_name, _ = os.path.splitext(filename)

        for lang in ['german', 'dutch', 'french']:
            text_file_path = os.path.join(texts_dir, lang, base_name + '.jpg.txt')
            if os.path.exists(text_file_path):
                try:
                    with open(text_file_path, 'r', encoding='utf-8') as tf:
                        texts[lang] = tf.read()
                except Exception as e:
                    print(f"Error reading {text_file_path}: {e}")
                    texts[lang] = ''
            else:
                texts[lang] = ''

        # Append data to lists
        raw_paths.append(raw_file_path)
        german_processed.append(processed_paths['german'])
        dutch_processed.append(processed_paths['dutch'])
        french_processed.append(processed_paths['french'])
        german_texts.append(texts['german'])
        dutch_texts.append(texts['dutch'])
        french_texts.append(texts['french'])

# Create a Polars DataFrame
df = pl.DataFrame({
    "raw": raw_paths,
    "german_processed": german_processed,
    "dutch_processed": dutch_processed,
    "french_processed": french_processed,
    "german_text": german_texts,
    "dutch_text": dutch_texts,
    "french_text": french_texts
})

# Preview the DataFrame
print(df.head())

# Optional: Save as a Parquet file for efficient storage
df.write_parquet(os.path.join(base_dir, "posters_data.parquet"))
print("Data saved as Parquet file.")

Zip file already exists. Skipping download.
Dataset already extracted.
shape: (5, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ raw          ┆ german_proc ┆ dutch_proce ┆ french_proc ┆ german_text ┆ dutch_text  ┆ french_text │
│ ---          ┆ essed       ┆ ssed        ┆ essed       ┆ ---         ┆ ---         ┆ ---         │
│ str          ┆ ---         ┆ ---         ┆ ---         ┆ str         ┆ str         ┆ str         │
│              ┆ str         ┆ str         ┆ str         ┆             ┆             ┆             │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ C:\Users\rom ┆ C:\Users\ro ┆ C:\Users\ro ┆             ┆ pt a        ┆ oe : ____   ┆             │
│ ai\Desktop\d ┆ mai\Desktop ┆ mai\Desktop ┆             ┆             ┆ hebben de   ┆             │
│ atatho…      ┆ \datatho…   ┆ \datatho…   ┆             ┆ AM oe       ┆ ec ingen 2… ┆             │
│     

In [1]:
import os
import gc
import torch
import polars as pl
import numpy as np

from transformers import pipeline
from sentence_transformers import SentenceTransformer
import os
import polars as pl
import requests
from zipfile import ZipFile

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# ---------------------------------------------
# Utility: chunk text so we don't exceed max tokens
# ---------------------------------------------
def chunk_text(text, chunk_size=512):
    """
    Splits text into smaller chunks while preserving words.
    """
    words = text.split()
    return [" ".join(words[i : i + chunk_size]) for i in range(0, len(words), chunk_size)]

# ---------------------------------------------
# 1) Language Detection
# ---------------------------------------------
def detect_language(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads the language detection pipeline once,
    sets the validation message if the detected language != expected.
    """
    # The known codes for each language
    expected_language_codes = {
        "german": "de",
        "dutch": "nl",
        "french": "fr"
    }
    
    print(f">> Loading language detection model  ...")
    lang_detector = pipeline(
        "text-classification",
        model="papluca/xlm-roberta-base-language-detection"
    )
    df_dicts = df.to_dicts()
    for lang in ['dutch', 'german', 'french']:
      for row in df_dicts:
          text = row.get(f"{lang}_text", "").strip()
          if not text:
              row[f"{lang}_text_validation"] = ""
              continue
          
          try:
              detected = lang_detector(text[:512])[0]["label"].lower()  # partial text is enough
          except Exception as e:
              print(f"Language detection error on row: {e}")
              detected = ""
          
          # If the detected code != the expected code, flag it
          expected_code = expected_language_codes.get(lang, "")
          if detected != expected_code:
              row[f"{lang}_text_validation"] = f"text in {lang} is not the right one"
          else:
              row[f"{lang}_text_validation"] = ""
    
    # Unload pipeline
    del lang_detector
    gc.collect()
    torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)

# ---------------------------------------------
# 2) Text Improvement
# ---------------------------------------------
def improve_text(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads the text-generation model once, only improves text
    if the previous validation said it's the correct language.
    """
    improvement_prompt = {
        "german": (
            "Du bist ein OCR-Korrektor. Korrigiere nur OCR-Fehler und ändere sonst nichts. "
            "Gib ausschließlich den korrigierten Text zurück, ohne jegliche Kommentare oder Erklärungen. "
            "Korrigiere nur die OCR-Fehler in dem Text, den der Benutzer senden wird. "
            "Antworte NUR mit dem komplett korrigierten Text, ohne jegliche Erläuterungen oder Kommentare."
        ),
        "dutch": (
            "Je bent een OCR-corrector. Corrigeer uitsluitend OCR-fouten en verander verder niets. "
            "Corrigeer alleen de OCR-fouten in de tekst die de gebruiker zal sturen. "
            "Geef uitsluitend de volledig gecorrigeerde tekst terug, zonder enige toelichting."
        ),
        "french": (
            "Vous êtes un correcteur OCR. Ne corrigez que les erreurs d'OCR et ne modifiez rien d'autre. "
            "Corrigez uniquement les erreurs d’OCR dans le texte que l’utilisateur va envoyer. "
            "Répondez exclusivement avec la version intégralement corrigée, sans aucune explication ni commentaire."
        ),
    }
    
    models={'german':'meta-llama/Llama-3.2-1B-Instruct','dutch':'utter-project/EuroLLM-1.7B-Instruct','french':'meta-llama/Llama-3.2-1B-Instruct'}

    print(f">> Loading text-improvement model ...")
    #imp_pipeline = pipeline("text-generation", model="BSC-LT/salamandra-2b-instruct") #pipeline("text-generation", model="stabilityai/stablelm-2-1_6b")
    df_dicts = df.to_dicts()
    for lang in ['german', 'dutch', 'french']:
      imp_pipeline = pipeline("text-generation", model=models[lang])
      for row in df_dicts:
          validation = row.get(f"{lang}_text_validation", "")
          original_text = row.get(f"{lang}_text", "").strip()

          # If text is empty or we already flagged the language as incorrect, skip improvement
          if not original_text or "not the right one" in validation:
              row[f"improved_{lang}_text"] = original_text
              continue

          # Otherwise, correct the text
          chunks = chunk_text(original_text, chunk_size=512)
          improved_pieces = []
          for chunk in chunks:
            prompt = [{"role": "system", "content": improvement_prompt[lang]},
                        {"role": "user", "content": chunk}]
            try:
                result = imp_pipeline(prompt, max_new_tokens=256)
                  # The pipeline might echo the prompt, so remove it if present
                full_gen = result[-1]["generated_text"][-1]['content']
                  # For safety: remove any leading copy of 'prompt'
                #cleaned = full_gen.replace(prompt, "").strip()
                improved_pieces.append(full_gen if full_gen else chunk)
            except Exception as e:
                  print(f"Improvement error: {e}")
                  improved_pieces.append(chunk)
          print(" ".join(improved_pieces))
          row[f"improved_{lang}_text"] = " ".join(improved_pieces)

      # Unload pipeline
      del imp_pipeline
      gc.collect()
      torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)

# ---------------------------------------------
# 3) Sentiment Analysis
# ---------------------------------------------
def get_sentiment_score(text,sentiment_pipeline, chunk_size=300):
    """
    Computes sentiment score by chunking text and averaging star ratings (1-5).
    """
    chunks = chunk_text(text, chunk_size)
    scores = []
    for chunk in chunks:
        try:
            result = sentiment_pipeline(chunk, truncation=True, max_length=512)
            label = result[0]['label']  # e.g. '4 stars'
            scores.append(int(label.split()[0]))
        except:
            pass
    return sum(scores)/len(scores) if scores else None

def analyze_sentiment(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads the sentiment pipeline once, applies to improved texts,
    then unloads.
    """
    print(f">> Loading sentiment model ...")
    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model="nlptown/bert-base-multilingual-uncased-sentiment"
    )

    df_dicts = df.to_dicts()
    for lang in ['dutch', 'german', 'french']:
      for row in df_dicts:
          text = row.get(f"improved_{lang}_text", "").strip()
          if not text:
              row[f"sentiment_{lang}"] = ""
              continue
          
          score = None
          try:
              # We reuse the chunk-based function:
              def local_chunker(txt):
                  for subchunk in chunk_text(txt, 300):
                      yield subchunk

              all_scores = []
              for sub in local_chunker(text):
                  result = sentiment_pipe(sub, truncation=True, max_length=512)
                  label = result[0]['label']
                  all_scores.append(int(label.split()[0]))
              if all_scores:
                  score = sum(all_scores)/len(all_scores)
          except Exception as e:
              print(f"Sentiment error: {e}")
              score = None
          
          if score is not None:
              row[f"sentiment_{lang}"] = f"{score:.2f} stars"
          else:
              row[f"sentiment_{lang}"] = ""

    # Unload pipeline
    del sentiment_pipe
    gc.collect()
    torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)

# ---------------------------------------------
# 4) Named Entity Recognition
# ---------------------------------------------
def extract_time_location(text):
    """
    Extracts time and location entities using NER.
    """
    if not text.strip():
        return "", ""
    results = ner_pipe(text)
    
    times = {ent['word'] for ent in results if 'TIME' in ent.get('entity', '').upper()}
    locs  = {ent['word'] for ent in results if 'LOC' in ent.get('entity', '').upper()}
    return " ".join(times), " ".join(locs)

def recognize_entities(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads the NER pipeline once, applies to improved text, unloads after.
    """
    print(f">> Loading NER model...")
    global ner_pipe
    ner_pipe = pipeline(
        "token-classification",
        model="Babelscape/wikineural-multilingual-ner"
    )

    df_dicts = df.to_dicts()
    for lang in ['dutch', 'german', 'french']:
      for row in df_dicts:
          text = row.get(f"improved_{lang}_text", "").strip()
          if not text:
              row[f"extracted_time_{lang}"] = ""
              row[f"extracted_location_{lang}"] = ""
              continue

          t, loc = extract_time_location(text)
          row[f"extracted_time_{lang}"] = t
          row[f"extracted_location_{lang}"] = loc

    # Unload pipeline
    del ner_pipe
    gc.collect()
    torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)

# ---------------------------------------------
# 5) Zero-Shot Topic Classification
# ---------------------------------------------
WW1_LABELS = [
    "military orders","martial law","war propaganda","German army decrees",
    "anti-resistance measures","civilian resistance","spying and informants",
    "penalties and punishments","executions","arrests and detentions",
    "occupation policies","curfews","travel restrictions","collaboration",
    "civilian cooperation","deportations","censorship","press restrictions",
    "public information control","war economy","wartime taxation","forced labor",
    "food supply","rationing","price controls","black market suppression",
    "public safety","health regulations","refugee management","forced evacuations",
    "civilian trials","public executions","house searches","checkpoint controls",
    "German language imposition","educational censorship","public gathering bans"
]

def zero_shot_topics(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads a zero-shot classification model once, applies to each row,
    and unloads.
    """
    print(f">> Loading zero-shot model ...")
    zshot_pipeline = pipeline(
        "zero-shot-classification",
        model="joeddav/xlm-roberta-large-xnli",
        force_download=True
    )
    
    df_dicts = df.to_dicts()
    for lang in ['dutch', 'german', 'french']:
      for row in df_dicts:
          text = row.get(f"improved_{lang}_text", "").strip()
          if not text:
              row[f"topics_{lang}_text"] = ""
              continue
          
          try:
              result = zshot_pipeline(
                  text,
                  candidate_labels=WW1_LABELS,
                  multi_label=True
              )
              extracted = [
                  lbl
                  for lbl, score in zip(result["labels"], result["scores"])
                  if score > 0.5
              ]
              row[f"topics_{lang}_text"] = ", ".join(extracted)
          except Exception as e:
              print(f"Zero-shot error: {e}")
              row[f"topics_{lang}_text"] = ""

    # Unload pipeline
    del zshot_pipeline
    gc.collect()
    torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)

# ---------------------------------------------
# 6) Embeddings
# ---------------------------------------------
def compute_embeddings(df: pl.DataFrame) -> pl.DataFrame:
    """
    Loads the SentenceTransformer once, encodes each improved text, unloads.
    """
    print(f">> Loading embedding model ...")
    emb_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    df_dicts = df.to_dicts()
    for lang in ['dutch', 'german', 'french']:
      for row in df_dicts:
          text = row.get(f"improved_{lang}_text", "").strip()
          if not text:
              row[f"{lang}_embedding"] = []
              continue
          try:
              emb = emb_model.encode(text).tolist()
              row[f"{lang}_embedding"] = emb
          except Exception as e:
              print(f"Embedding error: {e}")
              row[f"{lang}_embedding"] = []

    # Unload model
    del emb_model
    gc.collect()
    torch.cuda.empty_cache()

    return pl.DataFrame(df_dicts)


# ---------------------------------------------
#                  Main Flow
# ---------------------------------------------
def process_language(df: pl.DataFrame) -> pl.DataFrame:
    """
    Runs each step in the desired order for a specific language
    and returns the updated DataFrame.
    1) Language detection
    2) Improve text only if correct
    3) Sentiment
    4) NER
    5) Zero-shot classification
    6) Embeddings
    """
    print(f"===== Processing =====")
    df = detect_language(df)
    df = improve_text(df)
    df = analyze_sentiment(df)
    df = recognize_entities(df)
    df = zero_shot_topics(df)
    df = compute_embeddings(df)
    return df


if __name__ == "__main__":
  df = pl.read_parquet(r"C:\Users\romai\Desktop\datathon\Posters\posters_data.parquet")
  df = process_language(df)
  df.write_parquet("preprocessed_df.parquet")

c:\Users\romai\Desktop\datathon\Datathon_KUL\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


===== Processing =====
>> Loading language detection model  ...
>> Loading text-improvement model ...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


pt a AM oe _finden die Kontrollen der in der Gemeinde = : wohnenden mannlichen Belgier, die in den Jahren 1885 bis 1898 geboren sind, der Mitglieder der ehemaligen Garde civique, der in ihre Heimat. - entlassenen a Mag Oe feindlichen_ Kriegsteilnehmer und der feindlichen Auslander statt. : Die Kontrolle erfolgt in : | ee oe, um Uhr vormittags (deutsche Zeit). Den Anordnungen der Wachmannschaften und der bei der Kontrolle anwesenden belgischen Polizei- und Gemeindebeamten ist — unbedingt Folge zu leisten. Die anwesenden Truppen sind angewiesen, im Bedarfsfalle von den Waffen Gebrauch zu machen. : Die Kontr ollpflichtigen unterstehen am Kontrolltage d den deutschen Kriegsgesetzen. — a Bei der Oniollversimanlung ist Rauchen, sowie das Tragen von : Abzeichen verboten, ca seas he: andere Storung der Offentlichen Ordnung. Der Verka


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

AVI5S 
Dispositions réglementaires de Varrété du 31 juillet 1917, concernant la saisie et la livraison obligatoire des objets installation en cuivre, laiton et bronze.

Article 1e, Obligation de livrer.
En vertu de l’article 5 de l’arrété H. G. A. 22928 pris le 4 juillet 1917 par Son Excellence M. le Gouverneur général en vue d’étendre la saisie et la livraison obligatoire des objets d’installation en cuivre, laiton et bronze se trouvant dans les ménages ainsi que l’intérieur ou l’extérieur des batiments et conformément à l’article 5, 3° et 4° alinéas, de l’arrété H. G. A. 24895 du 31 juillet 1917, tous les intéressés (voir § 7 de l’arrété C. C. IV. A. 23418 du 13 décembre 1916) sont mis en demeure de livrer les objets rentrant dans les catégories visées par l’article batiments et jardins publics et dont la déclaration n’était pas exigée (en vertu de l’article 4, 4° alinéa, dudit arrêté), leur poids ne dépassant pas 20 kg. par classe; 1g Rampes d’escalier ;. (2) 20. Barres d’appui et g

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Arret concernant la déclaration et le commerce du linge de lit, de ménage ou de table. Article 4er, Objets visés par l’arrété, Doivent étre déclarés, conformément aux dispositions spéciales du présent arrété : Tous les stocks de linge de lit, de ménage ou de table de toute espéce, blanc ou de couleur, non usagé, en usage ou usagé, destiné 4 recouvrir les objets de literie ou les lits dans les pensions, pensionnats, asiles, etc. ou 6tre utilisés, d’une fagon quelconque, dans les hétels et restaurants, ménages et cuisines, notamment les tates de toute espéce, les couvertures, les draps, les essuie-mains, les nappes et napperonn, les serviettes de ménage ou autres et les torchons de toute espéce, du moment que lesdits stocks se trouvent dans le territoire du Gouvernement général le jour de la publication du présent arrété (jour du relevé). Article 2. Personnes vis 4, les dispositions du présent arrêté sont erronées, car il est écrit "arrété" au lieu de "arrêté". 

4, les dispositions du p

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


>> Loading embedding model ...


In [ ]:
%pip install huggingface_hub

In [ ]:
df = pl.read_parquet("preprocessed_df.parquet")

In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# Store extracted embeddings
embedding_data = []

languages = ['french', 'dutch', 'german']

for lang in languages:
    emb_col = f"{lang}_embedding"
    
    # Efficiently extract embeddings using iter_rows()
    for row in df.iter_rows(named=True):
        emb = row[emb_col]
        if emb is not None:  # Ensure we have an embedding
            embedding_data.append({
                "embedding": np.array(emb),  # Convert to NumPy array
                "language": lang
            })

# Convert to Polars DataFrame
emb_df = pl.DataFrame(embedding_data)

# Convert embeddings into NumPy array for t-SNE
X = np.vstack(emb_df["embedding"].to_list())  # Extract NumPy arrays from Polars

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)

# Add t-SNE dimensions back to Polars DataFrame
emb_df = emb_df.with_columns([
    pl.Series("tsne_1", X_tsne[:, 0]),
    pl.Series("tsne_2", X_tsne[:, 1])
])

# Convert to Pandas for visualization
emb_df_pandas = emb_df.to_pandas()

# Plot using Seaborn
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=emb_df_pandas,
    x="tsne_1",
    y="tsne_2",
    hue="language",
    palette="Set2",
    s=80,
    alpha=0.8
)
plt.title("2D Visualization of Embeddings by Language")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend(title="Language")
plt.show()

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Function to extract a numeric sentiment from a label like "4.20 stars"
def extract_sentiment_numeric(s):
    m = re.search(r'(\d+(\.\d+)?)', s)
    if m:
        return float(m.group(1))
    else:
        return None

# Create numeric sentiment columns for each language (we use French as our main example here)
df['sentiment_french_numeric'] = df['sentiment_french'].apply(extract_sentiment_numeric)
df['sentiment_dutch_numeric'] = df['sentiment_dutch'].apply(extract_sentiment_numeric)
df['sentiment_german_numeric'] = df['sentiment_german'].apply(extract_sentiment_numeric)

### 1. Analyze Sentiment by Extracted Time (French)
# Group by the extracted_time_french column and compute the average sentiment.
# (Note: extracted_time_french is a string; in a real scenario, you might need to parse dates.)
time_group = df.groupby('extracted_time_french')['sentiment_french_numeric'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=time_group, x='extracted_time_french', y='sentiment_french_numeric', palette="Blues_d")
plt.title("Average French Sentiment by Extracted Time")
plt.xlabel("Extracted Time")
plt.ylabel("Average Sentiment (stars)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2. Analyze Sentiment by Extracted Location (French)
loc_group = df.groupby('extracted_location_french')['sentiment_french_numeric'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=loc_group, x='extracted_location_french', y='sentiment_french_numeric', palette="Greens_d")
plt.title("Average French Sentiment by Extracted Location")
plt.xlabel("Extracted Location")
plt.ylabel("Average Sentiment (stars)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3. Analyze Topics and Their Relation to Sentiment (French)
# Assume that topics_french_text is a comma-separated string of topics.
topic_list = []
for topics in df['topics_french_text']:
    if topics and isinstance(topics, str):
        for topic in topics.split(','):
            topic = topic.strip()
            if topic:
                topic_list.append(topic)

# Count frequency of each topic.
topic_counts = Counter(topic_list)
topic_df = pd.DataFrame(topic_counts.items(), columns=['topic', 'count'])
# Take top 10 topics by frequency.
top_topics = topic_df.sort_values(by='count', ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(data=top_topics, x='topic', y='count', palette="Oranges_d")
plt.title("Top 10 French Topics by Frequency")
plt.xlabel("Topic")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Now, compute average sentiment per topic.
topic_sentiments = {}
for idx, row in df.iterrows():
    sentiment = extract_sentiment_numeric(row['sentiment_french'])
    topics = row['topics_french_text']
    if topics and sentiment is not None:
        for topic in topics.split(','):
            topic = topic.strip()
            if topic:
                topic_sentiments.setdefault(topic, []).append(sentiment)

# Compute the average sentiment for each topic.
topic_avg_sentiment = {topic: sum(vals)/len(vals) for topic, vals in topic_sentiments.items()}
topic_sentiment_df = pd.DataFrame(topic_avg_sentiment.items(), columns=['topic', 'avg_sentiment'])

# Merge with frequency to filter for top topics by frequency.
topic_sentiment_df = topic_sentiment_df.merge(topic_df, on='topic')
# Filter to top 10 topics (by frequency)
top_topic_sentiment = topic_sentiment_df.sort_values(by='count', ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(data=top_topic_sentiment, x='topic', y='avg_sentiment', palette="Purples_d")
plt.title("Average French Sentiment by Top 10 Topics")
plt.xlabel("Topic")
plt.ylabel("Average Sentiment (stars)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
%pip install wordcloud

In [ ]:
%python -m spacy download fr_core_news_sm
%python -m spacy download nl_core_news_sm
%python -m spacy download de_core_news_sm

In [ ]:
import spacy
import re
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Load spaCy models for French, Dutch, and German.
nlp_fr = spacy.load("fr_core_news_sm")
nlp_nl = spacy.load("nl_core_news_sm")
nlp_de = spacy.load("de_core_news_sm")

def preprocess_text(text, language):
    """
    Preprocess the text using spaCy:
      - Lowercase
      - Remove punctuation
      - Remove stopwords
      - Lemmatize tokens
    """
    # Select the appropriate spaCy model
    if language == "french":
        nlp = nlp_fr
    elif language == "dutch":
        nlp = nlp_nl
    elif language == "german":
        nlp = nlp_de
    else:
        return text.lower()  # fallback

    # Process the text with spaCy.
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        # Use the lemma and ensure it's lowercase and stripped of extra whitespace.
        lemma = token.lemma_.lower().strip()
        if lemma:
            tokens.append(lemma)
    return " ".join(tokens)

languages = ['french', 'dutch', 'german']

for lang in languages:
    # Combine all improved texts for the current language.
    text_data = " ".join(df[f"improved_{lang}_text"].dropna().tolist())

    # Preprocess the text thoroughly.
    processed_text = preprocess_text(text_data, lang)

    # Generate the word cloud.
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color="white",
        collocations=False,
        stopwords=STOPWORDS  # additional stopwords can be added if needed
    ).generate(processed_text)

    # Plot the word cloud.
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"WordCloud for {lang.capitalize()}")
    plt.show()